In [1]:
# pulling from Oracle transaction database at work -- I ran it once, you should read 2 cells down to pull from local drive
import cx_Oracle
import pandas.io.sql as sql
import yaml
import os 

cred = yaml.load(open(os.path.expanduser('prod_cred.yml')))
conn = cx_Oracle.connect(cred['USER'] + '/' + cred['PW'] + '@PROD')
base = sql.read_sql(
    """
    select assessed_land_value, assessed_total_value, assessed_imp_value, 
        zip, 
        full_bathroom_count + partial_bathroom_count * 0.5 bath,
        bedroom_count as bed,
        2009 - year_built as age,
        stories,
        garage_type,
        parking,
        total_units,
        pool,
        fireplace,
        property_type,
        lot_size,
        fidelity_latitude,
        fidelity_longitude
        
        from bdr.fidelity_assessment_record f
        JOIN bdr.zipcode_city_mapping zcm ON f.zip = zcm.zipcode
        WHERE zcm.mls_table = 'MLS_CA_BA'
        and assessment_year = 2009
        and lot_size > 0
    """, conn)

base.to_csv(path_or_buf = 'base.csv')

In [ ]:
from sklearn.cross_validation import train_test_split

base_l, base_s = train_test_split(base, test_size = 1000, random_state = 32)
base_s = pd.DataFrame(base_s, columns = base.columns)
base_s.to_csv('base_s.csv')

#READ THIS
The full data (base.csv) is too big for GitHub, so I sample 1000 random rows into base_s.csv (using train_test_split) with the step above. If using the full data (not on GitHub), run everything below; if using the sample data, adjust the cell below to read from base_s.csv instead of base.csv.

For most applications, do not run the 2 cells above this one.

In [3]:
import pandas as pd

base = pd.read_csv('base.csv')
print len(base)
base.head()

1558278


C:\Users\dfeng\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\io\parsers.py:1130: DtypeWarning: Columns (8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,ASSESSED_LAND_VALUE,ASSESSED_TOTAL_VALUE,ASSESSED_IMP_VALUE,ZIP,BATH,BED,AGE,STORIES,GARAGE_TYPE,PARKING,TOTAL_UNITS,POOL,FIREPLACE,PROPERTY_TYPE,LOT_SIZE,FIDELITY_LATITUDE,FIDELITY_LONGITUDE
0,0,23503,9688,33191,94607,1.0,2,119,1,G,0,0,NaN,NaN,NaN,6450,37.812862,-122.284366
1,1,51918,121142,173060,94607,2.0,6,118,2,NaN,0,2,NaN,NaN,NaN,4690,37.812487,-122.280142
2,2,35703,24221,59924,94610,1.5,2,87,1,G,0,0,NaN,NaN,NaN,3666,37.813370,-122.246670
3,3,217500,507500,725000,94610,1.5,4,94,1.5,G,0,0,NaN,NaN,NaN,5120,37.816322,-122.248903
4,4,190200,443800,634000,94610,1.5,3,84,2,G,0,0,NaN,NaN,NaN,3150,37.817159,-122.249869


In [4]:
base = base.drop('Unnamed: 0', axis = 1)
base.head().T.to_csv('data_example.csv')

In [5]:
base.describe().T

,count,mean,std,min,25%,50%,75%,max
ASSESSED_LAND_VALUE,1558278,173641.223270,306787.064111,0.000000,46616.000000,101761.000000,206400.000000,1.134974e+08
ASSESSED_TOTAL_VALUE,1558278,204622.388653,239538.804009,0.000000,83766.000000,160509.000000,261788.500000,7.808417e+07
ASSESSED_IMP_VALUE,1558278,378263.611922,471608.242032,0.000000,144973.250000,281639.000000,478592.750000,1.134974e+08
ZIP,1558278,94742.482704,392.571872,93109.000000,94534.000000,94597.000000,95051.000000,9.600100e+04
BATH,1558278,1.985419,0.946865,0.000000,1.000000,2.000000,2.500000,2.000000e+01
BED,1558278,2.880011,1.392383,0.000000,2.000000,3.000000,4.000000,2.000000e+01
AGE,1466427,44.196920,24.995731,0.000000,25.000000,43.000000,58.000000,1.990000e+02
PARKING,1558278,1.272550,11.772471,0.000000,0.000000,1.000000,2.000000,2.700000e+03
TOTAL_UNITS,1558278,0.215387,0.933769,0.000000,0.000000,0.000000,0.000000,3.000000e+02
LOT_SIZE,1558278,6783.750177,40797.599576,1.000000,3637.000000,5950.000000,7841.000000,4.250000e+07


In [6]:
# how bad are the zeros?

for i in base.columns:
    print i + ': ' + str((base[i] == 0).sum())

ASSESSED_LAND_VALUE: 1694
ASSESSED_TOTAL_VALUE: 41902
ASSESSED_IMP_VALUE: 1669
ZIP: 0
BATH: 98502
BED: 180468
AGE: 486
STORIES: 0
GARAGE_TYPE: 0
PARKING: 650849
TOTAL_UNITS: 1314170
POOL: 0
FIREPLACE: 0
PROPERTY_TYPE: 0
LOT_SIZE: 0
FIDELITY_LATITUDE: 9081
FIDELITY_LONGITUDE: 9081


In [7]:
base = base[base.ASSESSED_TOTAL_VALUE > 0] # remove Land
base = base[base.FIDELITY_LATITUDE > 0] # remove properties with no lat/long information
base = base[base.ASSESSED_LAND_VALUE > 0] # remove properties with no land value

In [8]:
for i in base.columns:
    print i + ': ' + str((base[i] == 0).sum())

ASSESSED_LAND_VALUE: 0
ASSESSED_TOTAL_VALUE: 0
ASSESSED_IMP_VALUE: 0
ZIP: 0
BATH: 57837
BED: 139695
AGE: 444
STORIES: 0
GARAGE_TYPE: 0
PARKING: 609447
TOTAL_UNITS: 1271159
POOL: 0
FIREPLACE: 0
PROPERTY_TYPE: 0
LOT_SIZE: 0
FIDELITY_LATITUDE: 0
FIDELITY_LONGITUDE: 0


Large # of properties with no bath, no bed, have to check if it's right or if it's studio.

In [9]:
len(base) - base.count() # how many errors?

ASSESSED_LAND_VALUE           0
ASSESSED_TOTAL_VALUE          0
ASSESSED_IMP_VALUE            0
ZIP                           0
BATH                          0
BED                           0
AGE                       52146
STORIES                  460074
GARAGE_TYPE              364663
PARKING                       0
TOTAL_UNITS                   0
POOL                    1383176
FIREPLACE               1298910
PROPERTY_TYPE           1442674
LOT_SIZE                      0
FIDELITY_LATITUDE             0
FIDELITY_LONGITUDE            0
dtype: int64

In [10]:
base.PROPERTY_TYPE.value_counts()

SFR    41594
PUD    11742
CND     9240
MFD     6290
RES     1662
2ND      908
MFG       24
COM        1
MOH        1
AGR        1
dtype: int64

PROPERTY_TYPE does not look useful, the missing data appears to be non-trivial

With AGE, I'll assume missing data = 0 or avg or see/look up how the algorithm handles it.<br>
With STORIES, I'll assume missing data = 1<br>
With GARAGE_TYPE, POOL and FIERPLACE, I'll assume missing data = None (or Unknown if None turns out to be common)